In [2]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
%matplotlib inline

In [3]:
#Reading input files
df = pd.read_csv("Documents/JK/churn_prediction/train.csv")
df_test = pd.read_csv("Documents/JK/churn_prediction/test.csv")

/anaconda/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Define a generic function using Pandas replace function
def coding(col, codeDict):
    colCoded = pd.DataFrame(col, copy=True)
    for key, value in codeDict.items():
        colCoded.replace(key, value, inplace=True)
    return colCoded

In [5]:
#Change categories to numeric values
df[["FINAL_WORTH_prev1","ENGAGEMENT_TAG_prev1"]] = coding(df[["FINAL_WORTH_prev1","ENGAGEMENT_TAG_prev1"]], {'NO':0,'LOW':1,'MEDIUM':2,'HIGH':3})
df.rename(columns = {'FINAL_WORTH_prev1' : 'FINAL_WORTH', 'ENGAGEMENT_TAG_prev1' : 'ENGAGEMENT_TAG_LATEST'}, inplace=True)

In [6]:
df_test[["FINAL_WORTH_prev1","ENGAGEMENT_TAG_prev1"]] = coding(df_test[["FINAL_WORTH_prev1","ENGAGEMENT_TAG_prev1"]], {'NO':0,'LOW':1,'MEDIUM':2,'HIGH':3})
df_test.rename(columns = {'FINAL_WORTH_prev1' : 'FINAL_WORTH', 'ENGAGEMENT_TAG_prev1' : 'ENGAGEMENT_TAG_LATEST'}, inplace=True)

In [7]:
df['OCCUP_ALL_NEW'].replace(['RETIRED','NON_INDIVIDUA','MISSING'],'OTHERS',inplace=True)
df_test['OCCUP_ALL_NEW'].replace(['RETIRED','NON_INDIVIDUA','MISSING'],'OTHERS',inplace=True)

In [8]:
#Exploratory Data Analysis
#Checking sample size
print df.shape, df_test.shape

(300000, 377) (200000, 376)


In [9]:
#Print all rows and columns. Dont hide any
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [10]:
#Back up for transformation - later on
df_bkp = df
df_test_bkp = df_test

In [13]:
df['brn_code'] = df['brn_code'].apply(str)
df_test['brn_code'] = df_test['brn_code'].apply(str)

In [14]:
#Remove ID fields and separate Target
df_Y = df['Responders']
df = df.drop(['Responders','UCIC_ID'], axis =1)

In [34]:
#Filter columns into numerical & categorical
df_num = df.select_dtypes(include=[np.number]) #Numbers which are categories must be verified
df_cat = df.select_dtypes(exclude=[np.number])
print df_num.shape, df_cat.shape

(300000, 338) (300000, 37)


In [35]:
#Function to check % of missing values
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    return mis_val_table_ren_columns

In [36]:
#Function to eliminate variables with more than 99% missing values - need function 'missing_values_table'
def non_missing_col(df):
    df_col = missing_values_table(df) #Calc missing values for each column
    df_col = df_col.loc[df_col['% of Total Values'] <= 99] #Filter dataframe with less missing values
    df_col = df_col.index.values.tolist() #Convert dataframe to columns list
    return df_col

In [37]:
#Eliminate Categorical Variables with more than 99% missing values
df_cat_col = missing_values_table(df_cat)
df_elim_col = df_cat_col.loc[df_cat_col['% of Total Values'] > 99]
df_elim_col = df_elim_col.index.values.tolist()
df_elim = df_cat[df_elim_col]

In [38]:
#Verify whether eliminated categorical fields have 100% correlation
df_elim = df_elim.join(df_Y)
print df_elim.shape

(300000, 16)


In [39]:
#Numerical variables with LT 99% missing values
df_num_col = non_missing_col(df_num)
df_num = df_num[df_num_col]
#Categorical variables with LT 99% missing values
df_cat_col = non_missing_col(df_cat)
df_cat = df_cat[df_cat_col]
print df_num.shape, df_cat.shape

(300000, 281) (300000, 22)


In [40]:
#Find Count of distinct values in categorical fields
count = df_cat.apply(pd.Series.nunique)
dstnct_cnt_col = pd.DataFrame(count, columns=['count'])
# dstnct_cnt_col

In [41]:
#Eliminate Variables by intuition
df_cat_col.remove('city') #Not suitable for prediction - too many values
df_cat_col.remove('brn_code') #Not suitable for prediction - too many values
df_cat = df_cat[df_cat_col]

In [42]:
#Find Count of distinct values in categorical fields
count = df_cat.apply(pd.Series.nunique)
dstnct_cnt_col = pd.DataFrame(count, columns=['count'])
# dstnct_cnt_col

In [44]:
#Binary Variables(1 or 2)
df_binary_col = dstnct_cnt_col.loc[dstnct_cnt_col['count'] <=2]
df_binary_col = df_binary_col.index.values.tolist()
df_binary = df_cat[df_binary_col]
#Nominal Variables(2+)
df_nominal_col = dstnct_cnt_col.loc[dstnct_cnt_col['count'] >2]
df_nominal_col = df_nominal_col.index.values.tolist()
df_nominal = df_cat[df_nominal_col]

In [49]:
df = df_bkp

In [50]:
#Eliminate Variables by intuition
df_num_col.remove('zip') #Not suitable for prediction - too many values
df_num = df_num[df_num_col]
df_binary_col.remove('FRX_PrevQ1_N') #Not suitable for prediction - single value

In [55]:
df_test = df_test_bkp

In [56]:
Cust_id_test = df_test[['UCIC_ID']]

In [58]:
print df.shape, df_test.shape

(300000, 377) (200000, 376)


In [59]:
#Training_Validation Split
df_train, df_validation = train_test_split(df,test_size=0.20,random_state=190)
print "Train shape : ",df_train.shape
print "Validation shape : ",df_validation.shape

Train shape :  (240000, 377)
Validation shape :  (60000, 377)


In [60]:
#training Target
train_Y = df_train ['Responders']
validation_Y = df_validation ['Responders']
print train_Y.shape, validation_Y.shape

(240000,) (60000,)


In [61]:
df_train = df_train.drop(['Responders'], axis =1)
df_validation = df_validation.drop(['Responders'], axis =1)
print "Train shape : ",df_train.shape
print "Validation shape : ",df_validation.shape

Train shape :  (240000, 376)
Validation shape :  (60000, 376)


In [62]:
df_train_num = df_train[df_num_col]
df_validation_num = df_validation[df_num_col]
df_test_num = df_test[df_num_col]

In [63]:
print df_train_num.shape, df_validation_num.shape, df_test_num.shape

(240000, 280) (60000, 280) (200000, 280)


In [64]:
#Impute median values for numerical variables
def impute_median(df):
    fill_NaN = Imputer(missing_values=np.nan, strategy='median', axis=1)
    imputed_df = pd.DataFrame(fill_NaN.fit_transform(df))
    imputed_df.columns = df.columns
    imputed_df.index = df.index
    return imputed_df

In [65]:
imputed_df_train_num = impute_median(df_train_num)
imputed_df_validation_num = impute_median(df_validation_num)
imputed_df_test_num = impute_median(df_test_num)

In [66]:
print imputed_df_train_num.shape, imputed_df_validation_num.shape, imputed_df_test_num.shape

(240000, 280) (60000, 280) (200000, 280)


In [68]:
df_nominal_col

['HNW_CATEGORY',
 'OCCUP_ALL_NEW',
 'Charges_cnt_PrevQ1_N',
 'RBI_Class_Audit',
 'gender_bin']

In [69]:
#Handling Categorical variables
df_train_nominal = df_train[df_nominal_col]
df_validation_nominal = df_validation[df_nominal_col]

In [70]:
# df_test_cat = df_test[df_cat_col]
df_test_nominal = df_test[df_nominal_col]

In [71]:
# print df_test_cat.shape, df_test_nominal.shape

In [72]:
def impute_mode(df):
    imputed_df = df.apply(lambda x:x.fillna(x.value_counts().index[0]))
    return imputed_df

In [73]:
imputed_df_train_nominal = impute_mode(df_train_nominal)
imputed_df_validation_nominal = impute_mode(df_validation_nominal)
imputed_df_test_nominal = impute_mode(df_test_nominal)

In [74]:
#Function to creating dummy variables for Nominal fields for a set of columns
def create_dummy_set(df_col,df):
    for i in df_col:
        dummies = pd.get_dummies(df[i]).rename(columns=lambda x: i+'_'+ str(x))
        #Adding to input variables
        df = pd.concat([df, dummies], axis=1)
        #Dropping column without having to reassign
        df.drop([i], inplace=True, axis=1)
        print df.shape
    return df

In [75]:
df_train_nominal = create_dummy_set(df_nominal_col,imputed_df_train_nominal)
df_validation_nominal = create_dummy_set(df_nominal_col,imputed_df_validation_nominal)
df_test_nominal = create_dummy_set(df_nominal_col,imputed_df_test_nominal)

(240000, 7)
(240000, 12)
(240000, 15)
(240000, 18)
(240000, 20)
(60000, 7)
(60000, 12)
(60000, 15)
(60000, 18)
(60000, 20)
(200000, 7)
(200000, 12)
(200000, 15)
(200000, 18)
(200000, 20)


In [76]:
df_train_binary = df_train[df_binary_col]
df_validation_binary = df_validation[df_binary_col]
df_test_binary = df_test[df_binary_col]

In [77]:
df_train_binary = create_dummy_set(df_binary_col,df_train_binary)
df_validation_binary = create_dummy_set(df_binary_col,df_validation_binary)
df_train_binary = df_train_binary[df_train_binary.columns.drop(list(df_train_binary.filter(regex='MISSING')))]
df_validation_binary = df_validation_binary[df_validation_binary.columns.drop(list(df_validation_binary.filter(regex='MISSING')))]

(240000, 14)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 15)
(240000, 16)
(240000, 17)
(60000, 14)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 15)
(60000, 16)
(60000, 17)


In [78]:
df_test_binary = create_dummy_set(df_binary_col,df_test_binary)
df_test_binary = df_test_binary[df_test_binary.columns.drop(list(df_test_binary.filter(regex='MISSING')))]

(200000, 14)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 15)
(200000, 16)
(200000, 17)


In [79]:
df_test_binary.head(5)

,EMAIL_UNSUBSCRIBE_Y,EFT_SELF_TRANSFER_PrevQ1_N,EFT_SELF_TRANSFER_PrevQ1_Y,AL_TAG_LIVE_Y,CC_TAG_LIVE_Y,DEMAT_TAG_LIVE_Y,HL_TAG_LIVE_Y,SEC_ACC_TAG_LIVE_Y,INS_TAG_LIVE_Y,MF_TAG_LIVE_Y,OTHER_LOANS_TAG_LIVE_Y,RD_TAG_LIVE_Y,FD_TAG_LIVE_Y,Billpay_Active_PrevQ1_N_B_1,Billpay_Reg_ason_Prev1_N_B_1
0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1


In [80]:
df_train_binary.drop('EFT_SELF_TRANSFER_PrevQ1_N', axis=1, inplace=True)
df_validation_binary.drop('EFT_SELF_TRANSFER_PrevQ1_N', axis=1, inplace=True)
df_test_binary.drop('EFT_SELF_TRANSFER_PrevQ1_N', axis=1, inplace=True)

In [81]:
df_train = imputed_df_train_num.join(df_train_nominal).join(df_train_binary)
print df_train.shape

(240000, 314)


In [82]:
df_validation = imputed_df_validation_num.join(df_validation_nominal).join(df_validation_binary)
print df_validation.shape

(60000, 314)


In [83]:
df_test = imputed_df_test_num.join(df_test_nominal).join(df_test_binary)
print df_test.shape

(200000, 314)


In [84]:
print df_train.shape, df_validation.shape, df_test.shape

(240000, 314) (60000, 314) (200000, 314)


In [ ]:
_xgb = xgb.XGBClassifier(objective='binary:logistic',max_depth=5,n_estimators=1000,colsample_bytree=.75,learning_rate=0.06).fit(df_train, train_Y,eval_metric='auc')
train_pred = _xgb.predict_proba(df_train)
val_pred = _xgb.predict_proba(df_validation)

In [93]:
train_pred_val = _xgb.predict(df_train)
val_pred_val = _xgb.predict(df_validation)
print recall_score(train_Y, train_pred_val)
print recall_score(validation_Y, val_pred_val)
print confusion_matrix(train_Y, train_pred_val)
print confusion_matrix(validation_Y, val_pred_val)

0.56566976384
0.492815121998
[[193775   4897]
 [ 17950  23378]]
[[47856  1775]
 [ 5259  5110]]


In [97]:
test_pred = _xgb.predict_proba(df_test)

In [98]:
df_test_pred = pd.DataFrame(test_pred.tolist())

In [99]:
predicted_result = Cust_id_test.join(df_test_pred)

In [100]:
predicted_result = predicted_result.rename(columns = {0 : 'Non-Responders', 1 : 'Responders'})

In [101]:
predicted_result = predicted_result.drop('Non-Responders', 1)

In [102]:
predicted_result.shape

(200000, 2)

In [103]:
predicted_result.head(10)

,UCIC_ID,Responders
0,337734,0.306379
1,488166,0.027382
2,410785,0.071905
3,389145,0.098597
4,221090,0.164572
5,2651,0.013117
6,344310,0.045364
7,53881,0.024969
8,499665,0.065219
9,482927,0.042110


In [104]:
predicted_result.to_csv("Documents/JK/churn_prediction/BOG_SB_output.csv",sep=',',index=False)